In [1]:
#| default_exp consultas

In [1]:
#| export
import re
from typing import Union, Iterable

import requests
import pandas as pd

from fastcore.basics import store_attr
from fastcore.foundation import L
from fastcore.parallel import parallel
from fastcore.xtras import Path
from fastcore.script import call_parse, Param

BASEURL = "http://webservicesintranet{}.anatel.gov.br/receita/rest/"
AMBIENTE = {'ds', 'hm', 'su', 'pd'}
TYPE = {'cpf': "obterPessoaFisica", 'cnpj': "obterPessoaJuridica"}
CPF_SIMPLES = ('cpf', 'nome', 'nomeMae', 'dataNascimento', 'estrangeiro',
               'tituloEleitor', 'dataAtualizacao', 'dataRegistroAnatel', 
               'resultado', 'unidadeAdministrativaCodigo', 'anoObito', 'erro')
SEXO =  {1: 'Masculino', 2: 'Feminino', 9 : 'Não encontrado'}
SIZE = {11: 'cpf', 14: 'cnpj'}

In [2]:
#| hide
from nbdev.showdoc import *

## Validador de CPF | CPNJ 

In [3]:
def validador_cpf(cpf: str) -> bool:
    """
    Valida um CPF
    """
    assert len(cpf) == SIZE['cpf'], f"CPF inválido, digite um CPF válido com {SIZE['cpf']} dígitos"    

## Classe Principal para Requisição

In [4]:
#| export 
class Request:
    def __init__(self, 
                 cpf_usr: Union[str, int], # CPF do usuário requisitante
                 ambiente: str = 'ds', # Ambiente onde realizar a requisição: ds: desenvolvimento, hm: homologação, su: sustentação, pd: produção
                 origem: str = None, # Texto com identificação da requisição: e.g. "Teste"
                 cache: int = 36, # Tempo de expiração do cache em meses (Produção)
    ):
        """Classe para fazer requisições ao Web Service Infoconv da Receita Federal"""
        store_attr()
        assert isinstance(self.origem, str), "origem não pode ficar vazio e deve ser uma string de até 30 caracteres"
        self.origem = self.origem[:30]
        assert (ambiente := self.ambiente.lower()) in AMBIENTE, f"Ambiente inválido, escolha uma das opções {AMBIENTE}"
        if ambiente == 'pd':
            self.ambiente = ''

    def format_url(self, query)-> str: 
        if (tipo := len(query)) not in SIZE:
            raise ValueError("Query inválida, CPF possui 11 caracteres e CNPJ possui 14 caracteres")
        tipo = SIZE[tipo] 
        req_type = TYPE[tipo]
        suffix = ''
        if self.cache is not None:
            try:
                cache = int(self.cache)
                assert cache >= 0, "Tempo de expiração do cache inválido, escolha um número inteiro maior que zero"
            except ValueError as e:
                raise ValueError("Valor inválido de expira_cache, escolha um número inteiro maior que zero") from e
            req_type += 'IgnoraCacheAntigo'
            suffix = f'&mesesExpiraCache={cache}'

        return f'{BASEURL.format(self.ambiente)}{req_type}?{tipo}={query}&cpfUsuario={self.cpf_usr}{suffix}&origem={self.origem}'

    @staticmethod
    def parse_json(json_dict)-> dict:
        d = {}
        for k,v in json_dict.items():
            if isinstance(v, list): 
                v = '|'.join(v)
            if isinstance(v, dict):
                d.update({f'{k}_{sk}': sv for sk, sv in v.items()})
                continue
            d[k] = v
        return {k:v.lstrip().rstrip() if isinstance(v, str) else v for k, v in d.items()}

    def get_request(self, url)-> dict:
        """Make a request to the url and return the json response"""
        r = requests.get(url)
        if r.status_code == 200 and r.headers['content-type'] == 'application/json':
            return Request.parse_json(r.json())
        return {}

    def query_ws(self, 
                 query: Union[str, int], # Identificador da requisição: CPF ou CNPJ de acordo com o tipo

                ):
        query = ''.join(re.findall("\d", query))
        return self.get_request(self.format_url(query))
    
    def batch_request(self, queries: Iterable[Union[str, int]]):                        
        return L(queries).map(self.query_ws)
        

## Testes

In [5]:
from fastcore.test import ExceptionExpected, test_eq
cpf_usuario = '31888916877'
identificador = '31888916877'
origem = "Teste REST API"
ambiente = 'su'

In [6]:
req = Request(cpf_usuario, ambiente, origem)

In [7]:
req.query_ws(identificador)

{'cpf': '31888916877',
 'nome': 'Ronaldo da Silva Alves Batista',
 'situacaoCadastral_codigo': 0,
 'situacaoCadastral_valor': 'Regular',
 'paisResidencia_residenteExterior': False,
 'paisResidencia_codigoPais': 0,
 'paisResidencia_nomePais': '',
 'nomeMae': 'MARIA ELIANE DA SILVA BATISTA',
 'dataNascimento': '1985-04-02',
 'sexo_codigo': 1,
 'sexo_valor': 'Masculino',
 'ocupacao_naturezaOcupacaoCodigo': 22,
 'ocupacao_naturezaOcupacaoDescricao': 'Servidor público de autarquia ou fundação federal',
 'ocupacao_ocupacaoPrincipalCodigo': 116,
 'ocupacao_ocupacaoPrincipalDescricao': 'Servidor das demais carreiras da administração pública direta, autárquica e fundacional 117 Titular de Cartório',
 'ocupacao_exercicioOcupacao': 2018,
 'endereco_logradouro': 'Rua Vergueiro',
 'endereco_numero': '3106',
 'endereco_complemento': 'Apto 13',
 'endereco_cep': '04102001',
 'endereco_bairro': 'Vila Mariana',
 'endereco_codigoMunicipio': 3550308,
 'endereco_nomeMunicipio': 'SAO PAULO',
 'endereco_uf':

In [8]:
cpfs = req.batch_request(['31888916877', '76538672868'])
cpfs

(#2) [{'cpf': '31888916877', 'nome': 'Ronaldo da Silva Alves Batista', 'situacaoCadastral_codigo': 0, 'situacaoCadastral_valor': 'Regular', 'paisResidencia_residenteExterior': False, 'paisResidencia_codigoPais': 0, 'paisResidencia_nomePais': '', 'nomeMae': 'MARIA ELIANE DA SILVA BATISTA', 'dataNascimento': '1985-04-02', 'sexo_codigo': 1, 'sexo_valor': 'Masculino', 'ocupacao_naturezaOcupacaoCodigo': 22, 'ocupacao_naturezaOcupacaoDescricao': 'Servidor público de autarquia ou fundação federal', 'ocupacao_ocupacaoPrincipalCodigo': 116, 'ocupacao_ocupacaoPrincipalDescricao': 'Servidor das demais carreiras da administração pública direta, autárquica e fundacional 117 Titular de Cartório', 'ocupacao_exercicioOcupacao': 2018, 'endereco_logradouro': 'Rua Vergueiro', 'endereco_numero': '3106', 'endereco_complemento': 'Apto 13', 'endereco_cep': '04102001', 'endereco_bairro': 'Vila Mariana', 'endereco_codigoMunicipio': 3550308, 'endereco_nomeMunicipio': 'SAO PAULO', 'endereco_uf': 'SP', 'telefone_

In [9]:
pd.DataFrame(cpfs)

,cpf,nome,situacaoCadastral_codigo,situacaoCadastral_valor,paisResidencia_residenteExterior,paisResidencia_codigoPais,paisResidencia_nomePais,nomeMae,dataNascimento,sexo_codigo,...,endereco_uf,telefone_ddd,telefone_numero,unidadeAdministrativaCodigo,anoObito,estrangeiro,dataAtualizacao,dataRegistroAnatel,resultado,erro
0,31888916877,Ronaldo da Silva Alves Batista,0,Regular,False,0,,MARIA ELIANE DA SILVA BATISTA,1985-04-02,1,...,SP,0011,00000000,0819600,0,False,2018-09-24,2020-01-19,CPF encontrado,
1,76538672868,Antonio Alves Batista,0,Regular,False,0,,MARIA ALVES BATISTA,1948-01-15,1,...,SP,0000,00000000,0811300,0,False,2018-09-14,2020-01-19,CPF encontrado,


In [12]:
req.query_ws('00280273000137')

{'cnpj': '00280273000137',
 'matriz': True,
 'nomeEmpresarial': 'Samsung Eletronica da Amazonia Ltda',
 'nomeFantasia': 'Samsung da Amazonia',
 'situacaoCadastral_codigo': '02',
 'situacaoCadastral_valor': 'Ativa',
 'situacaoCadastral_dataAlteracao': '2005-11-03T00:00:00',
 'naturezaJuridica_codigo': '2062',
 'naturezaJuridica_descricao': 'SOCIEDADE EMPRESARIA LIMITADA',
 'dataAbertura': '1994-10-31',
 'cnaePrincipal': '2640000',
 'cnaeSecundario': '1922599|2621300|2622100|2632900|2670102|3313999|4618402|4618499|4645101|4649401|4649402|4649499|4651601|4651602|4652400|5211799|6202300|6203100|7490199|7739002|7739099|8211300|9511800|9512600|9521500|9529199|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|0000000|00

In [13]:
req.query_ws('02.030.715/0001-12')

{'cnpj': '02030715000112',
 'matriz': True,
 'nomeEmpresarial': 'Agencia Nacional de Telecomunicacoes',
 'nomeFantasia': 'Anatel',
 'situacaoCadastral_codigo': '02',
 'situacaoCadastral_valor': 'Ativa',
 'situacaoCadastral_dataAlteracao': '1998-07-28T00:00:00',
 'naturezaJuridica_codigo': '1104',
 'naturezaJuridica_descricao': 'AUTARQUIA FEDERAL',
 'dataAbertura': '1997-08-12',
 'cnaePrincipal': '8413200',
 'endereco_logradouro': 'Quadra Saus Quadra 6 Blocos C,e,f e H',
 'endereco_numero': '10',
 'endereco_complemento': 'Andar: Bloco H;',
 'endereco_cep': '70070940',
 'endereco_bairro': 'Asa Sul',
 'endereco_codigoMunicipio': 5300108,
 'endereco_nomeMunicipio': 'Brasilia',
 'endereco_uf': 'DF',
 'telefone1_ddd': '61',
 'telefone1_numero': '23122134',
 'telefone2_ddd': '61',
 'telefone2_numero': '23121953',
 'email': 'catty@anatel.gov.br',
 'responsavel_cpf': '00757367135',
 'responsavel_nome': 'Carlos Manuel Baigorri',
 'capitalSocial': '0',
 'porte_codigo': 5,
 'porte_valor': 'Demais'

In [ ]:
# %%time
# cnpjs = req.batch_request(50 * ['02.030.715/0001-12', '00280273000137'])

CPU times: total: 484 ms
Wall time: 23.2 s


In [ ]:
# with ExceptionExpected(ex=AssertionError, regex= 'origem não pode ficar vazio e deve ser uma string de até 30 caracteres'):
#     format_url(cpf_usuario=cpf_usuario, identificador=identificador)

In [ ]:
# with ExceptionExpected(ex=AssertionError, regex= 'origem não pode ficar vazio e deve ser uma string de até 30 caracteres'):
#     format_url(cpf_usuario=cpf_usuario, identificador=identificador, origem = 50)

In [ ]:
# with ExceptionExpected(ex=AssertionError, regex="Ambiente inválido"):
#     format_url(cpf_usuario=cpf_usuario, identificador=identificador, origem='Teste', ambiente='ts')

In [ ]:
# origem = 'Teste'
# for ambiente in AMBIENTE:
#     url = format_url(cpf_usuario=cpf_usuario, identificador=identificador, origem=origem, ambiente=ambiente)
#     test_eq(url, f'http://webservicesintranet{ambiente}.anatel.gov.br/receita/rest/obterPessoaFisicaIgnoraCacheAntigo?cpf={identificador}&cpfUsuario={cpf_usuario}&mesesExpiraCache=36&origem={origem}')

In [ ]:
# from itertools import product
# origem = 'Teste'
# expira_cache = 3
# for (ambiente, tipo) in product(AMBIENTE, TYPE):
#     url = format_url(cpf_usuario=cpf_usuario, 
#                      identificador=identificador, 
#                      origem=origem, 
#                      ambiente=ambiente, 
#                      tipo=tipo, 
#                      expira_cache=expira_cache)
#     test_eq(url, f'http://webservicesintranet{ambiente}.anatel.gov.br/receita/rest/{TYPE[tipo]}IgnoraCacheAntigo?{tipo}={identificador}&cpfUsuario={cpf_usuario}&mesesExpiraCache={expira_cache}&origem={origem}')

In [ ]:
# from itertools import product
# origem = 'Teste'
# expira_cache = None
# for (ambiente, tipo) in product(AMBIENTE, TYPE):
#     url = format_url(cpf_usuario=cpf_usuario, 
#                      identificador=identificador, 
#                      origem=origem, 
#                      ambiente=ambiente, 
#                      tipo=tipo,
#                     expira_cache=expira_cache,
#                     )
#     test_eq(url, f'http://webservicesintranet{ambiente}.anatel.gov.br/receita/rest/{TYPE[tipo]}?{tipo}={identificador}&cpfUsuario={cpf_usuario}&origem={origem}')

In [18]:
#| export
def save_request(results: Iterable, # Lista com o retorno das requisições
                 saida: str, # Nome do Arquivo de Saída
):
    """Salva a lista de requisições `results` no arquivo `saida`"""
    df = pd.DataFrame(results)
    try:
        saida = Path(saida)
    except TypeError:
        saida = Path.cwd()
    match suffix := saida.suffix:
        case '.csv' | '.txt':
            df.to_csv(saida, index=False)
        case '.xlsx':
            df.to_excel(saida, index=False, engine='openpyxl')
        case '.json':
            df.to_json(saida)
        case '.md':
            df.to_markdown(saida, index=False)
        case '.html':
            df.to_html(saida, index=False)
        case _:
            df.to_clipboard(sep=',', index=False)

@call_parse
def request_from_file(filename: Param("Arquivo texto de entrada: 1 CPF | CNPJ por linha", str),
                      cpf_usuario: Param("CPF do usuário requisitante", str),
                      ambiente: Param("Ambiente onde realizar a requisição: ds | hm | su | pd", str) = 'ds', 
                      origem: Param("Texto com identificação da requisição: e.g. 'Teste'", str) = None, 
                      cache: Param("Tempo de expiração do cache em meses (Produção)",  int) = 36, 
                      saida: Param("Arquivo de saída da requisição", str) = None, 
):
    """Lê o arquivo `filename` com um CPF | CPNJ por linha. Faz a requisição no `ambiente` do receita-ws e salva os resultados em `saida`"""
    contents = Path(filename).readlines()
    req = Request(cpf_usuario, ambiente, origem, cache)
    results = req.batch_request(contents)
    save_request(results, saida)

In [19]:
#| hide
import nbdev; nbdev.nbdev_export()